In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import random

In [5]:
cs = 4                         #number of candidate charging station

In [6]:
trip_information = pd.read_csv("Trip_information_NumericalStudy.csv")
terminal_distance = pd.read_csv("Between_terminal_distance_NumericalStudy.csv")
##terminals
#1---University Medical Center
#2---Salt Lake Central Station
#3---This is the Place State Park
#4---North Temple Station

In [6]:
trip_information

,Trips,Start_time,End_time,Start_terminal,End_terminal,Duration,Energy_need
0,1,315,345,2,1,0.500000,20.1750
1,2,345,378,1,2,0.550000,20.1750
2,3,345,375,2,1,0.500000,20.1750
3,4,371,403,4,3,0.533333,19.1528
4,5,375,408,1,2,0.550000,20.1750
...,...,...,...,...,...,...,...
111,112,1245,1276,2,1,0.516667,20.1750
112,113,1246,1282,1,2,0.600000,20.1750
113,114,1275,1306,2,1,0.516667,20.1750
114,115,1276,1312,1,2,0.600000,20.1750


In [7]:
terminal_distance

,Unnamed: 0,1,2,3,4,Depot
0,1,0.0,7.7,2.80,3.80,3.9
1,2,7.7,0.0,5.50,0.70,0.2
2,3,2.8,5.5,0.00,7.12,5.6
3,4,3.8,0.7,7.12,0.00,0.5


In [8]:
######function to calculate the travel time and energy needed between each trip and the depot
def depot_to_trip(trip_information, terminal_distance):
    from_depot_to_trip_time = []
    from_depot_to_trip_energy = []
    from_trip_to_depot_time = []
    from_trip_to_depot_energy = []
    i=0
    while i < 116:
        trip = trip_information.iloc[i]["Trips"]
        trip_starttime = trip_information.iloc[i]["Start_time"]
        trip_endtime = trip_information.iloc[i]["End_time"]
        trip_startterminal = trip_information.iloc[i]["Start_terminal"]
        trip_endterminal = trip_information.iloc[i]["End_terminal"]
        from_depot_distance = terminal_distance.iloc[int(trip_startterminal)-1]["Depot"]
        to_depot_distance = terminal_distance.iloc[int(trip_endterminal)-1]["Depot"]
        depot_to_trip_time_need = from_depot_distance/20
        trip_to_depot_time_need = to_depot_distance/20
        depot_to_trip_energy_need = from_depot_distance*2.69
        trip_to_depot_energy_need = to_depot_distance*2.69
        from_depot_to_trip_time.append(depot_to_trip_time_need)
        from_trip_to_depot_time.append(trip_to_depot_time_need)
        from_depot_to_trip_energy.append(depot_to_trip_energy_need)
        from_trip_to_depot_energy.append(trip_to_depot_energy_need)
        i += 1
    return from_depot_to_trip_time, from_trip_to_depot_time, from_depot_to_trip_energy, from_trip_to_depot_energy

In [9]:
from_depot_to_trip_time, from_trip_to_depot_time, from_depot_to_trip_energy, from_trip_to_depot_energy = depot_to_trip(trip_information, terminal_distance)

In [10]:
from_depot_to_trip_time = pd.DataFrame(from_depot_to_trip_time) 
from_trip_to_depot_time = pd.DataFrame(from_trip_to_depot_time)
from_depot_to_trip_energy = pd.DataFrame(from_depot_to_trip_energy)
from_trip_to_depot_energy = pd.DataFrame(from_trip_to_depot_energy)
from_depot_to_trip_time.to_csv("From_depot_to_trip_time")
from_trip_to_depot_time.to_csv("From_trip_to_depot_time")
from_depot_to_trip_energy.to_csv("From_depot_to_trip_energy")
from_trip_to_depot_energy.to_csv("From_trip_to_depot_energy")

In [7]:
From_depot_to_trip_time = pd.read_csv("From_depot_to_trip_time")
From_trip_to_depot_time = pd.read_csv("From_trip_to_depot_time")
From_depot_to_trip_energy = pd.read_csv("From_depot_to_trip_energy")
From_trip_to_depot_energy = pd.read_csv("From_trip_to_depot_energy")
print(From_depot_to_trip_time)
print(From_trip_to_depot_time)
print(From_depot_to_trip_energy)
print(From_trip_to_depot_energy)

     Unnamed: 0      0
0             0  0.010
1             1  0.195
2             2  0.010
3             3  0.025
4             4  0.195
..          ...    ...
111         111  0.010
112         112  0.195
113         113  0.010
114         114  0.195
115         115  0.195

[116 rows x 2 columns]
     Unnamed: 0      0
0             0  0.195
1             1  0.010
2             2  0.195
3             3  0.280
4             4  0.010
..          ...    ...
111         111  0.195
112         112  0.010
113         113  0.195
114         114  0.010
115         115  0.010

[116 rows x 2 columns]
     Unnamed: 0       0
0             0   0.538
1             1  10.491
2             2   0.538
3             3   1.345
4             4  10.491
..          ...     ...
111         111   0.538
112         112  10.491
113         113   0.538
114         114  10.491
115         115  10.491

[116 rows x 2 columns]
     Unnamed: 0       0
0             0  10.491
1             1   0.538
2             2 

In [12]:
From_depot_to_trip_energy.iloc[0][1]

0.538

In [13]:
######function to get the maximum available time for charging between trips and the energy needed between trips
def generate_between_trip(trip_information, terminal_distance):
    between_trip_time = pd.DataFrame()
    between_trip_energy = pd.DataFrame()
    i=0
    while i < 116:
        time_list = []
        energy_list = []
        j = 0
        while j < 116:
            trip1 = trip_information.iloc[i]["Trips"]
            trip2 = trip_information.iloc[j]["Trips"]
            trip1_starttime = trip_information.iloc[i]["Start_time"]
            trip2_starttime = trip_information.iloc[j]["Start_time"]
            trip1_endtime = trip_information.iloc[i]["End_time"]
            trip2_endtime = trip_information.iloc[j]["End_time"]
            trip1_startterminal = trip_information.iloc[i]["Start_terminal"]
            trip2_startterminal = trip_information.iloc[j]["Start_terminal"]
            trip1_endterminal = trip_information.iloc[i]["End_terminal"]
            trip2_endterminal = trip_information.iloc[j]["End_terminal"]
            between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
            between_time_need = between_distance/20
            between_energy_need = between_distance*2.69
            if trip1 == trip2:
                time_list.append(-1000)
                energy_list.append(-1000)
            else:
                if trip2_starttime < trip1_endtime + between_time_need*60:
                    time_list.append(-1000)
                    energy_list.append(-1000)
                else:
                    remain_time = (trip2_starttime - trip1_endtime)/60 - between_time_need
                    time_list.append(remain_time)
                    energy_list.append(between_energy_need)
            j += 1
        time_list = pd.DataFrame(time_list)
        energy_list = pd.DataFrame(energy_list)
        time_list = time_list.transpose()
        energy_list = energy_list.transpose()
        between_trip_time = between_trip_time.append(time_list)
        between_trip_energy = between_trip_energy.append(energy_list)
        i+=1
    return between_trip_time, between_trip_energy


In [14]:
result1, result2 = generate_between_trip(trip_information, terminal_distance)

In [15]:
result1.to_csv("Between_trip_time_NumericalStudy.csv") 
result2.to_csv("Between_trip_energy_NumericalStudy.csv")

In [8]:
available_charging_time = pd.read_csv("Between_trip_time_NumericalStudy.csv")
energy_need = pd.read_csv("Between_trip_energy_NumericalStudy.csv")

In [18]:
available_charging_time

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,106,107,108,109,110,111,112,113,114,115
0,0,-1000.0,0.0,-1000.0,0.243333,0.5,0.115,1.000,0.615,0.865000,...,13.850000,13.615000,14.016667,14.115000,14.516667,14.615000,15.016667,15.115000,15.516667,16.016667
1,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,0.065,0.450,0.700000,...,12.915000,13.450000,13.081667,13.950000,13.581667,14.450000,14.081667,14.950000,14.581667,15.081667
2,0,-1000.0,-1000.0,-1000.0,-1000.000000,0.0,-1000.000,0.500,0.115,0.365000,...,13.350000,13.115000,13.516667,13.615000,14.016667,14.115000,14.516667,14.615000,15.016667,15.516667
3,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,-1000.000,-1000.000,0.008333,...,12.743333,12.758333,12.910000,13.258333,13.410000,13.758333,13.910000,14.258333,14.410000,14.910000
4,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,-1000.000,-1000.000,0.200000,...,12.415000,12.950000,12.581667,13.450000,13.081667,13.950000,13.581667,14.450000,14.081667,14.581667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,0.000000,0.500000
112,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,0.015000
113,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,0.000000
114,0,-1000.0,-1000.0,-1000.0,-1000.000000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000


In [19]:
energy_need

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,106,107,108,109,110,111,112,113,114,115
0,0,-1000.0,0.0,-1000.0,10.222,0.0,20.713,0.000,20.713,20.713,...,0.000,20.713,0.000,20.713,0.000,20.713,0.000,20.713,0.000,0.000
1,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,20.713,0.000,0.000,...,20.713,0.000,20.713,0.000,20.713,0.000,20.713,0.000,20.713,20.713
2,0,-1000.0,-1000.0,-1000.0,-1000.000,0.0,-1000.000,0.000,20.713,20.713,...,0.000,20.713,0.000,20.713,0.000,20.713,0.000,20.713,0.000,0.000
3,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,-1000.000,-1000.000,14.795,...,7.532,14.795,7.532,14.795,7.532,14.795,7.532,14.795,7.532,7.532
4,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,-1000.000,-1000.000,0.000,...,20.713,0.000,20.713,0.000,20.713,0.000,20.713,0.000,20.713,20.713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000,...,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,0.000,0.000
112,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000,...,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,20.713
113,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000,...,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,0.000
114,0,-1000.0,-1000.0,-1000.0,-1000.000,-1000.0,-1000.000,-1000.000,-1000.000,-1000.000,...,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000,-1000.000


In [20]:
energy_need.iloc[0][4]

10.222

In [9]:
######generate following set
def generate_following_set(trip_information, terminal_distance):
    following_set_list = []
    i=0
    while i < 116:
        following_set = []
        j = 0
        while j < 116:
            trip1 = trip_information.iloc[i]["Trips"]
            trip2 = trip_information.iloc[j]["Trips"]
            trip1_starttime = trip_information.iloc[i]["Start_time"]
            trip2_starttime = trip_information.iloc[j]["Start_time"]
            trip1_endtime = trip_information.iloc[i]["End_time"]
            trip2_endtime = trip_information.iloc[j]["End_time"]
            trip1_startterminal = trip_information.iloc[i]["Start_terminal"]
            trip2_startterminal = trip_information.iloc[j]["Start_terminal"]
            trip1_endterminal = trip_information.iloc[i]["End_terminal"]
            trip2_endterminal = trip_information.iloc[j]["End_terminal"]
            between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
            between_time_need = between_distance/20
            if trip2_starttime >= trip1_endtime + between_time_need*60:
                following_set.append(int(trip2))
            j +=1
        following_set_list.append(following_set)
        i += 1
    return following_set_list

In [21]:
following_set_list = generate_following_set(trip_information, terminal_distance)

In [22]:
following_set_list

[[2,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 [7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  

In [25]:
dict_following_set = {}
for i in range(1,117):
    dict_following_set[i] = following_set_list[i-1]
dict_following_set

{1: [2,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 2: [7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  

In [29]:
import pickle
dict_following_set[2]


with open('Following_set_dict.pickle', 'wb') as handle:
    pickle.dump(dict_following_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
a = pickle.load(open('Following_set_dict.pickle','rb'))
a

{1: [2,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 2: [7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  

In [23]:
######define a function to get a following trip of a service trip
def return_following(i,following_set_list):
    following_set = following_set_list[i]
    if len(following_set) == 0:
        return -1
    else:
        following_set = following_set[0:5]
        return(random.choice(following_set))

In [24]:
######define a function to generate a block given a first trip
def block_generation(i,block,following_set_list):
    while True:
        m = return_following(i,following_set_list)
        if m != -1: 
            block.append(m)
            i = m
        else: 
            print("block=")
            print(block)
            return block

In [25]:
######define a function to get difference of two lists
def diff(list1, list2):
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1))))

In [26]:
######define a function to generate the service trip part of a chromosome
def trippart_chromosome_generation(rest_trips, chromosome, following_set_list, crosspoint, no_bus, no_trip):
    while True:
        if len(rest_trips) != 0:
            fist_trip = min(rest_trips)                        #get the trip with the earlest starting time in rest of the trips
            new_block = []
            new_block.append(fist_trip)
            new_block = block_generation(fist_trip,new_block,following_set_list)  #generate a new block
            rest_trips = diff(rest_trips, new_block)           #get rest of the trips
            chromosome = chromosome + new_block                #add the newly generated block to the chromosome
            no_bus += 1
            crosspoint.append(len(chromosome)) 
            no_trip.append(len(new_block))
            following_set_list = [[ele for ele in sub if ele not in chromosome] for sub in following_set_list]
        else:
            print("All trips have been assigned to blocks")
            print("Generated trip part of chromosome =")
            print(chromosome)
            crosspoint.pop()
            print("Possible crossover points =")
            print(crosspoint)
            print("Fleet size under this trip schedule =")
            print(no_bus)
            print("Number of trips for each bus =")
            print(no_trip)
            
            return chromosome, crosspoint, no_bus, no_trip

In [52]:
rest_trips = [i for i in range(117)]
rest_trips.pop(0)

0

In [53]:
chromosome = []
crosspoint = []
no_bus = 0
no_trip = []

In [54]:
trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, following_set_list, crosspoint, no_bus, no_trip)

block=
[1, 7, 18, 27, 36, 44, 49, 56, 66, 76, 82, 91, 99, 106, 112, 115]
block=
[2, 10, 21, 30, 38, 42, 51, 59, 67, 74, 87, 93, 98, 105, 109, 114, 116]
block=
[3, 5, 14, 25, 32, 45, 52, 57, 64, 75, 83, 95, 101, 110, 113]
block=
[4, 9, 16, 23, 28, 35, 40, 48, 60, 71, 79, 89, 97, 107]
block=
[6, 15, 22, 33, 47, 54, 61, 70, 80, 88, 103, 111]
block=
[8, 11, 19, 29, 37, 50, 58, 77, 81, 86, 90, 102, 108]
block=
[12, 24, 39, 63, 84, 94, 104]
block=
[13, 17, 34, 53, 69, 85, 96]
block=
[20, 41, 68, 100]
block=
[26, 55, 78, 92]
block=
[31, 72]
block=
[43, 62, 73]
block=
[46, 65]
All trips have been assigned to blocks
Generated trip part of chromosome =
[1, 7, 18, 27, 36, 44, 49, 56, 66, 76, 82, 91, 99, 106, 112, 115, 2, 10, 21, 30, 38, 42, 51, 59, 67, 74, 87, 93, 98, 105, 109, 114, 116, 3, 5, 14, 25, 32, 45, 52, 57, 64, 75, 83, 95, 101, 110, 113, 4, 9, 16, 23, 28, 35, 40, 48, 60, 71, 79, 89, 97, 107, 6, 15, 22, 33, 47, 54, 61, 70, 80, 88, 103, 111, 8, 11, 19, 29, 37, 50, 58, 77, 81, 86, 90, 102,

In [32]:
######generate the charging station part of the chromosome
Charging_station_part_chromosome = []
for i in range(cs):
    Charging_station_part_chromosome.append(random.randint(0,1))
print("Generated charging station part of chromosome =")
print(Charging_station_part_chromosome)

Generated charging station part of chromosome =
[1, 1, 1, 1]


In [55]:
trip_chromosome=trippart_chromosome[0]
crossover_point=trippart_chromosome[1]
fleet_size=trippart_chromosome[2]
trips_per_bus=trippart_chromosome[3]
print(trip_chromosome)
print(crossover_point)
print(fleet_size)
print(trips_per_bus)

[1, 7, 18, 27, 36, 44, 49, 56, 66, 76, 82, 91, 99, 106, 112, 115, 2, 10, 21, 30, 38, 42, 51, 59, 67, 74, 87, 93, 98, 105, 109, 114, 116, 3, 5, 14, 25, 32, 45, 52, 57, 64, 75, 83, 95, 101, 110, 113, 4, 9, 16, 23, 28, 35, 40, 48, 60, 71, 79, 89, 97, 107, 6, 15, 22, 33, 47, 54, 61, 70, 80, 88, 103, 111, 8, 11, 19, 29, 37, 50, 58, 77, 81, 86, 90, 102, 108, 12, 24, 39, 63, 84, 94, 104, 13, 17, 34, 53, 69, 85, 96, 20, 41, 68, 100, 26, 55, 78, 92, 31, 72, 43, 62, 73, 46, 65]
[16, 33, 48, 62, 74, 87, 94, 101, 105, 109, 111, 114]
13
[16, 17, 15, 14, 12, 13, 7, 7, 4, 4, 2, 3, 2]


In [57]:
######randomly assign a battery capacity for each bus in the bus fleet
bus_type = [1,2]                   #available bus type in the market
battery_type = [200,400]           #battery capacity of each bus type
battery_size = [0]*fleet_size      #define a list to store the battery size of each bus in the bus fleet
battery_lower = [0]*fleet_size     #define the lower bound of battery energy in bus
battery_upper = [0]*fleet_size     #define the upper bound of battery energy in bus
for i in range(fleet_size):        #randomly assign a battery capacity for each bus in the bus fleet
    battery_size[i] = random.choice(battery_type)
    battery_lower[i] =  battery_size[i]*0.2
    battery_upper[i] = battery_size[i]*0.8
print(battery_size)
print(battery_lower)
print(battery_upper)

[400, 200, 200, 400, 400, 400, 400, 400, 400, 400, 200, 200, 400]
[80.0, 40.0, 40.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 40.0, 40.0, 80.0]
[320.0, 160.0, 160.0, 320.0, 320.0, 320.0, 320.0, 320.0, 320.0, 320.0, 160.0, 160.0, 320.0]


In [58]:
######define the maximum charging power of a bus
p_bus_max = 350

In [59]:
######define the function to check feasibility of the generated chromosome
def check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chromosome):
    i = 0
    while (i < fleet_size):
        print("chromosome=")
        print(trip_chromosome)
        print("i=" +str(i))
        a = trips_per_bus[i]                        #get the number of trips for the bus i
        battery_l = battery_lower[i]                #get the battery capacity lower bound of the bus i
        battery_u = battery_upper[i]                #get the battery capacity upper bound of the bus i
        trip_chain = trip_chromosome[0:a]           #get the trip chain of the bus i
        b = trip_chain[0]                           #get the first trip in the trip chain of the bus i
        remain_energy = battery_u - From_depot_to_trip_energy.iloc[b-1][1] - trip_information.iloc[b-1]["Energy_need"]   #get the remaining energy in bus i after finishing its first trip
        if remain_energy < battery_l:
            print("The depot to first trip part of the trip chain "+str(i)+" is not feasible")
            break
        else:
            if a == 1:
                remain_energy = remain_energy - From_trip_to_depot_energy.iloc[b-1][1]
                if remain_energy < battery_l:
                    print("The only trip to depot part of the trip chain "+str(i)+" is not feasible")
                    break
                else:
                    print("Trip chain "+str(i)+" is feasible")
            elif a > 1:
                j = 1
                while (j < a):
                    print("j= "+str(j))
                    bb = trip_chain[j] 
                    print("bb= " +str(bb))
                    cc = trip_chain[j-1] 
                    print("cc= " +str(cc))
                    remain_energy = remain_energy - energy_need.iloc[cc-1][bb]
                    if remain_energy < battery_l:
                        print("The between trips part of the trip chain "+str(i)+" is not feasible")
                        break
                    else:
                        if j < a-1:
                            stt = trip_information.iloc[bb-1]["Start_terminal"]     #get the start terminal of the next trip
                            remain_energy = remain_energy + Charging_station_part_chromosome[int(stt)-1]*p_bus_max*available_charging_time.iloc[cc-1][bb]
                            if remain_energy > battery_u:
                                remain_energy = battery_u
                            else: 
                                remain_energy = remain_energy
                            remain_energy = remain_energy - trip_information.iloc[bb-1]["Energy_need"]
                            if remain_energy < battery_l:
                                print("The between trips part of the trip chain "+str(i)+" is not feasible")
                                break
                        else:
                            remain_energy = remain_energy - From_trip_to_depot_energy.iloc[bb-1][1]
                            if remain_energy < battery_l:
                                print("The last trip to depot part of the trip chain "+str(i)+" is not feasible")
                                break
                            else:
                                print("Trip chain "+str(i)+" is feasible")
                    j += 1
        trip_chromosome = trip_chromosome[a:]
        i += 1

In [60]:
check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chromosome)

chromosome=
[1, 7, 18, 27, 36, 44, 49, 56, 66, 76, 82, 91, 99, 106, 112, 115, 2, 10, 21, 30, 38, 42, 51, 59, 67, 74, 87, 93, 98, 105, 109, 114, 116, 3, 5, 14, 25, 32, 45, 52, 57, 64, 75, 83, 95, 101, 110, 113, 4, 9, 16, 23, 28, 35, 40, 48, 60, 71, 79, 89, 97, 107, 6, 15, 22, 33, 47, 54, 61, 70, 80, 88, 103, 111, 8, 11, 19, 29, 37, 50, 58, 77, 81, 86, 90, 102, 108, 12, 24, 39, 63, 84, 94, 104, 13, 17, 34, 53, 69, 85, 96, 20, 41, 68, 100, 26, 55, 78, 92, 31, 72, 43, 62, 73, 46, 65]
i=0
j= 1
bb= 7
cc= 1
j= 2
bb= 18
cc= 7
j= 3
bb= 27
cc= 18
j= 4
bb= 36
cc= 27
j= 5
bb= 44
cc= 36
j= 6
bb= 49
cc= 44
j= 7
bb= 56
cc= 49
j= 8
bb= 66
cc= 56
j= 9
bb= 76
cc= 66
j= 10
bb= 82
cc= 76
j= 11
bb= 91
cc= 82
j= 12
bb= 99
cc= 91
j= 13
bb= 106
cc= 99
j= 14
bb= 112
cc= 106
j= 15
bb= 115
cc= 112
Trip chain 0 is feasible
chromosome=
[2, 10, 21, 30, 38, 42, 51, 59, 67, 74, 87, 93, 98, 105, 109, 114, 116, 3, 5, 14, 25, 32, 45, 52, 57, 64, 75, 83, 95, 101, 110, 113, 4, 9, 16, 23, 28, 35, 40, 48, 60, 71, 79, 89, 97

In [61]:
feasible_chromosome = []
feasible_chromosome.append(Charging_station_part_chromosome)
feasible_chromosome.append(trip_chromosome)
feasible_chromosome.append(crossover_point)
fleet_size =[fleet_size]
feasible_chromosome.append(fleet_size)
feasible_chromosome.append(trips_per_bus)

feasible_chromosome = pd.DataFrame(feasible_chromosome)

In [62]:
feasible_chromosome

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7.0,18.0,27.0,36.0,44.0,49.0,56.0,66.0,76.0,...,55.0,78.0,92.0,31.0,72.0,43.0,62.0,73.0,46.0,65.0
2,16,33.0,48.0,62.0,74.0,87.0,94.0,101.0,105.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,17.0,15.0,14.0,12.0,13.0,7.0,7.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
feasible_chromosome.to_csv('Feasible chromosomes.csv', mode='a', index=None)

In [65]:
feasible_chromosome = pd.read_csv('Feasible chromosomes.csv')

In [66]:
feasible_chromosome

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7.0,18.0,27.0,36.0,44.0,49.0,56.0,66.0,76.0,...,55.0,78.0,92.0,31.0,72.0,43.0,62.0,73.0,46.0,65.0
2,16,33.0,48.0,62.0,74.0,87.0,94.0,101.0,105.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,17.0,15.0,14.0,12.0,13.0,7.0,7.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
feasible_chromosome.iloc[0][6]

nan

In [4]:
%gams_push fleet_size

KeyError: 'fleet_size'

In [1]:
%gams display fleet_size;
%gams_lst -e

UsageError: Line magic function `%gams` not found.
